### Python wrappers for invoking RNA structure abrrevation predicition methods RNAsnp and remuRNA
Author: Milad Miladi
For details about the methods please refer to their corrsponding documentations:

- https://www.ncbi.nlm.nih.gov/CBBresearch/Przytycka/index.cgi#remurna
- http://rth.dk/resources/rnasnp/

In [2]:
####     
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
import os
import pandas as pd
from tempfile import NamedTemporaryFile
from subprocess import Popen, PIPE
from StringIO import StringIO

#########################################
## Configuration
## Please set these two paths accordingly

REMURNA_PATH = '/home/milad/1Downloads/remuRNA/'
RNASNPPATH='/home/milad/1Downloads/RNAsnp-1.2/'

def run_RNAsnp(wild_fa, snp_tags, window=None):
    """
    A python wrapper invoking RNAsnp tool. 
    Please check RNAsnp documentation for details.
    
    Call example: run_RNAsnp('./wild.fa', ['G20C'])
    Parameters
    ----------
    wild_fa : str
        Fasta file name containing one RNA sequence 
    snp_tags : list
        Set of SNP tags required to be evaluatued on the input sequence
    window : int 
        If None, the RNAsnp window (-w) size. Windows larger than 800 will be passed as 800.

    Returns
    -------
    dataframe
        Pandas table of standard output

    """
    
    # Write SNP tags to a temporary file, TODO: Remove the temporary file?
    snp_file = NamedTemporaryFile(delete=False)
    snp_file.write('\n'.join(snp_tags))
    snp_file.close()
    
    if not os.path.isfile(wild_fa):
        raise RuntimeError ("Input fasta %s does not exist" % in_fasta_file)
    
    # Make a shell command line
    cmd = 'export RNASNPPATH="{}" ; {}/Progs/RNAsnp -f {} -s {} '.format(RNASNPPATH, RNASNPPATH, wild_fa, snp_file.name)
    if window is not None:
        if window > 800:
            print "WARNING RNAsnp window reduced to max possible: 800"
            window = 800
        cmd += '-w {}'.format(int(window))
    # print cmd
    p = Popen( cmd , stdin=PIPE, shell=True, stdout=PIPE, stderr=PIPE)
    out, err = p.communicate()
    if err:
        #raise RuntimeError("Error in calling RNAsnp\n{}\n{}\n".format(out, err))
        print "Error in calling RNAsnp\n{}\n{}\n".format(out, err)
    # print out
    out_cleaned = ""
    for line in out.split('\n'):
        if 'error' in line.lower():
            raise RuntimeError("RNASNP returned error for: {} message is:{}".format(wild_fa, line))
        elif 'warning' in line.lower():
            print ("ERROR: RNASNP returned warning for: {} message is:{}".format(wild_fa, line))
        else:
            out_cleaned += line+"\n"
            
    df_RNAsnp = pd.read_table(StringIO(out_cleaned))
    return  df_RNAsnp #.add_suffix('RNAsnp:')


def run_remuRNA(wild_fa, snp_tags, window=None):
    """
    A python wrapper invoking remuRNA tool. 
    Please check remuRNA documentation for details.
    Call example: run_remuRNA('./wild.fa', ['G20C'])
    Parameters
    ----------
    wild_fa : str
        Fasta file name containing one RNA sequence 
    snp_tags : list
        Set of SNP tags required to be evaluatued on the input sequence.
        Warning: remuRNA accepts only a single tag in each call.

    Returns
    -------
    dataframe
        Pandas table of standard output

    """
    assert(len(snp_tags)==1)
    if not os.path.isfile(wild_fa):
        raise RuntimeError ("Input fasta %s does not exist" % in_fasta_file)

    # Write RNA sequence and SNP tags to a temporary file, TODO: Remove the temporary file?
    tmp_fa = NamedTemporaryFile(suffix='.fa', delete=False)
    with open (wild_fa) as in_fa_handle:
        for line in in_fa_handle:
            tmp_fa.write(line)
    tmp_fa.write('\n'.join(['*'+tag for tag in snp_tags]))
    tmp_fa.close()
   
    # Make a shell command line
    cmd = REMURNA_PATH+'/remuRNA {} -p=3 '.format(tmp_fa.name)
    if window is not None:
        cmd += '-w={}'.format(int(window))
    # print cmd
    p = Popen( cmd , stdin=PIPE, shell=True, stdout=PIPE, stderr=PIPE)
    out, err = p.communicate()
    if err:
        raise RuntimeError("Error in calling remuRNA\n{}\n{}\n".format(out, err))

    # print out
    df = pd.read_table(StringIO(out))
    return  df

### Example calls

In [3]:
run_RNAsnp('./seq1.fa', ['U1013C'], window=200)

,SNP,w,Slen,GC,interval,d_max,p-value,interval.1,r_min,p-value.1
0,U1013C,200,3344,0.5411,975-1025,0.2432,0.0724,998-1052,0.0615,0.0932


In [6]:
run_remuRNA('./seq1.fa', ['U1013C'], window=200)

,SNP,MFE(wt),MFE(mu),dMFE,H(wt||mu),GCratio
0,T1013C,-56.466,-56.406,-0.06,2.317,0.541353


## Run RaSE

https://github.com/fabriziocosta/RaSE-RNA-Structural-Stability-Estimator

In [6]:
import os
import sys
rase_root_dir = '../../RaSE/RaSE_git/'
rase_src_dir = os.path.join(rase_root_dir, 'code')
sys.path = [rase_src_dir] + sys.path

eden_root_dir = '../../RaSE/EDeN/'
eden_src_dir = os.path.join(eden_root_dir)
sys.path = [eden_src_dir] + sys.path

from RaSE import make_fold, make_fold_vectorize
def run_RaSE(wild_seq, snp_tags, window=150, avg_bp_prob_cutoff=0.01,
                          hard_threshold=0.5,
                          max_num_edges=3,):
    assert(len(snp_tags)==1)
    import re
    
        

    matches = re.match(r'(\D)(\d+)(\D)', snp_tags[0])
    tag_tup = matches.groups()
    tag_tup = (tag_tup[0], int(tag_tup[1])-1, tag_tup[2]) 
    
    fold = make_fold(window_size=window,
                          max_bp_span=window-50,
                          avg_bp_prob_cutoff=avg_bp_prob_cutoff,
                          hard_threshold=hard_threshold,
                          max_num_edges=max_num_edges,
                          no_lonely_bps=True,
                          nesting=True)
    fold_vectorize = make_fold_vectorize(complexity=3, nbits=15, fold=fold)
    from RaSE import compute_SNP_stability
    score = compute_SNP_stability(wild_seq, snp_tag=tag_tup, fold_vectorize=fold_vectorize)
    print 'Rase-Score:', score
    return score

run_RaSE("ACGUGGCUG",['C2G'])

Rase-Score: 0.187300630503


0.18730063050346024